## 詞頻矩陣

In [9]:
import jieba
jieba.load_userdict('userdict.txt')
ary = ['富邦人壽首創保費1萬就能刷卡',
       '富邦人壽不老勇士Go來盃3對3籃賽南北開打',
       '富邦人壽不老勇士Go來盃開打']

In [10]:
corpus = []
for title in ary:
    corpus.append(' '.join(jieba.cut(title)))

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 

X = vectorizer.fit_transform(corpus)

In [12]:
X.shape

(3, 11)

In [13]:
vectorizer.get_feature_names()

['go', '不老', '人壽', '保費', '刷卡', '勇士', '南北', '富邦', '籃賽', '開打', '首創']

In [14]:
X.toarray()

array([[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]], dtype=int64)

## 相似度計算

In [17]:
a = '柯P 大巨蛋 遠雄'
b = '柯P 為了 大巨蛋 一案 找 遠雄 算帳'

#   柯P 大巨蛋 遠雄 為了 一案 找 算帳
# a  1    1    1    0    0   0   0
# b  1    1    1    1    1   1   1

In [25]:
import numpy as np
na = np.array([1,1,1,0,0,0,0])
nb = np.array([1,1,1,1,1,1,1])

np.sqrt(sum((nb - na) ** 2))

2.0

In [27]:
sum(na * nb) / (np.sqrt(sum(na ** 2)) * np.sqrt(sum(nb ** 2)) )

0.6546536707079772

In [23]:
a = '柯P 大巨蛋 遠雄'
b = '柯P 為了 大巨蛋 一案 找 遠雄 算帳 因為 遠雄 在 這案 中 出爾反爾'

#   柯P 大巨蛋 遠雄 為了 一案 找 算帳 因為 在 這案 中 出爾反爾
# a  1    1    1    0    0   0   0  0   0   0   0    0
# b  1    1    2    1    1   1   1  1   1   1   1    1

In [28]:
import numpy as np
na = np.array([1,1,1,0,0,0,0,0,0,0,0,0,0])
nb = np.array([1,1,2,1,1,1,1,1,1,1,1,1,1])

np.sqrt(sum((nb - na) ** 2))

3.3166247903553998

In [29]:
sum(na * nb) / (np.sqrt(sum(na ** 2)) * np.sqrt(sum(nb ** 2)) )

0.57735026918962584

In [31]:
import jieba
jieba.load_userdict('userdict.txt')
ary = ['富邦人壽首創保費1萬就能刷卡',
       '富邦人壽不老勇士Go來盃3對3籃賽南北開打',
       '富邦人壽不老勇士Go來盃開打']
corpus = []
for title in ary:
    corpus.append(' '.join(jieba.cut(title)))
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 

X = vectorizer.fit_transform(corpus)
X.toarray()

array([[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]], dtype=int64)

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(X)

array([[ 1.        ,  0.31622777,  0.4       ],
       [ 0.31622777,  1.        ,  0.79056942],
       [ 0.4       ,  0.79056942,  1.        ]])

In [33]:
from sklearn.metrics.pairwise import cosine_distances
cosine_distances(X)

array([[ 0.        ,  0.68377223,  0.6       ],
       [ 0.68377223,  0.        ,  0.20943058],
       [ 0.6       ,  0.20943058,  0.        ]])

## 新聞推薦引擎

In [45]:
import requests
from bs4 import BeautifulSoup
news_ary = []
for page in range(1,30):
    res = requests.get('https://tw.appledaily.com/new/realtime/{}'.format(page))
    soup = BeautifulSoup(res.text, 'html.parser')
    for rec in soup.select('.rtddt a'):
        dic = {}
        dic['link'] = rec['href']
        res2 = requests.get(dic['link'])
        soup2 = BeautifulSoup(res2.text, 'html.parser')
        dic['category'] = soup2.select('.current')[2].text
        dic['title']    = soup2.select_one('h1').text
        dic['content']  = soup2.select_one('.ndArticle_margin p').text
        news_ary.append(dic)
    print(page)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [46]:
import pandas
newsdf = pandas.DataFrame(news_ary)
newsdf.to_excel('appledaily20171214.xlsx')

In [48]:
import pandas
newsdf = pandas.read_excel('data/appledaily20171214.xlsx')

In [49]:
newsdf.head()

,category,content,link,title
0,政治,新增：立委說法民進黨立法院黨團預計在明天的院會中，讓改制農田水利會的《農田水利會組織通則》修...,https://tw.news.appledaily.com/politics/realti...,【更新】水利會改官派明闖關 綠委24小時前顧議場大門防藍突襲
1,論壇,邱俊棠／台中市民、見習醫師；曾任台灣醫學生聯合會對外副會長對於公民參與公眾事務而能得到單位首...,https://tw.news.appledaily.com/forum/realtime/...,請中市府為所當為 加速中火燃煤限制
2,社會,被控來台涉發展情報組織的中國學生周泓旭，因接觸我方外交部官員而露餡落網，今年9月被台北地院一...,https://tw.news.appledaily.com/local/realtime/...,陸生共諜嗆台司法 「不敢公開審理我」
3,生活,【廣編特輯】 對於全能神教會在中國所遭受的迫害與在韓國、香港及臺灣所面對的反面宣傳，非政府組...,https://tw.news.appledaily.com/life/realtime/2...,【特企】NGO聯名譴責中共迫害宗教信仰
4,生活,立法院交通委員會今審查《道路交通管理處罰條例》，通過營業大客車駕駛人行駛一般道路未繫安全帶，...,https://tw.news.appledaily.com/life/realtime/2...,乘客搭大客車不配合繫安全帶 立院初審通過可罰6千元


In [52]:
corpus = []
titles = []
for news in newsdf.iterrows():
    corpus.append(' '.join(jieba.cut(news[1]['content'])))
    titles.append(news[1]['title'])

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<899x38624 sparse matrix of type '<class 'numpy.int64'>'
	with 128131 stored elements in Compressed Sparse Row format>

In [67]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(X)
cs.shape

(899, 899)

In [65]:
na = np.array([1,9,2,6,8,4,5,7])
na.sort()
na

array([1, 2, 4, 5, 6, 7, 8, 9])

In [66]:
na = np.array([1,9,2,6,8,4,5,7])
na.argsort()

array([0, 2, 5, 6, 3, 7, 4, 1], dtype=int64)

In [72]:
rank = cs[0].argsort()[::-1]
for rec in rank[0:10]:
    if cs[0][rec] > 0.3:
        print(titles[rec], cs[0][rec])

【更新】水利會改官派明闖關　綠委24小時前顧議場大門防藍突襲 1.0
反對水利會改制　吳敦義下令：藍委做好夜宿立院抗爭準備 0.367845249572
農田水利會改公務機關　蔡英文：這不是綁樁 0.327587152559


In [73]:
def getSimiliarArticle(pos):
    rank = cs[pos].argsort()[::-1]
    for rec in rank[0:10]:
        similiarities = cs[pos][rec]
        if similiarities > 0.3:
            print(titles[rec], similiarities)

In [79]:
getSimiliarArticle(6)

喬妹升格宋太太首上工　現身中國見習近平 1.0
素顏依舊完美　喬妹升格人妻赴中開工 0.31870904608


## 同義詞

In [16]:
synonym_dic = {
    '柯P': '柯文哲'
}
s = '柯P為了大巨蛋一案找遠雄算帳'
' '.join([synonym_dic.get(ele, ele) for ele in jieba.cut(s)])


'柯文哲 為 了 大巨蛋 一案 找遠 雄算帳'

## 保險常見問題集

In [81]:
import requests
from bs4 import BeautifulSoup
res = requests.get('https://my83.com.tw/knowledge/client')
soup = BeautifulSoup(res.text, 'html.parser')

In [91]:
qalist =  soup.select('.accordionList')
questions = [qa.select('dt')[0].text for qa in qalist]
answers   = [qa.select('dd')[0].text for qa in qalist]
qa_ary = []
for q, a in zip(questions, answers):
    dic = {}
    dic['question'] = q
    dic['answer']   = a
    qa_ary.append(dic)

In [93]:
import pandas
df = pandas.DataFrame(qa_ary)
df.to_excel('my83qa.xlsx')

In [95]:
import pandas
issuarance = pandas.read_excel('my83qa.xlsx')

## 自動問答

In [124]:
q = '殘廢險的功用?'
q_seg = ' '.join(jieba.cut(q))

In [125]:
corpus    = [q_seg]
questions = [q]
answers   = ['NONE']
for qa in issuarance.iterrows():
    corpus.append(' '.join(jieba.cut(qa[1]['question'])))
    questions.append(qa[1]['question'])
    answers.append(  qa[1]['answer'])

In [126]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<16x65 sparse matrix of type '<class 'numpy.int64'>'
	with 82 stored elements in Compressed Sparse Row format>

In [127]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(X)

In [128]:
rank = cs[0].argsort()[::-1]
qa_dic = {}
qa_dic['question'] = questions[rank[1]]
qa_dic['answer']   = answers[rank[1]]
qa_dic['similarity']   = cs[0][rank[1]]
qa_dic

{'answer': '殘廢險能提供完整的殘廢保障，不像意外險只能理賠因意外所造成的殘廢，殘廢險能保障疾病及意外所造成的傷害，並且能避免保險公司對於殘廢原因的認定爭議。事實上，統計有6成的殘廢主因是疾病而非意外。',
 'question': 'Q：殘廢險的主要功用是什麼？',
 'similarity': 0.70710678118654746}

In [131]:
def getSimiliarAnswer(q):
    q_seg = ' '.join(jieba.cut(q))
    corpus    = [q_seg]
    questions = [q]
    answers   = ['NONE']
    for qa in issuarance.iterrows():
        corpus.append(' '.join(jieba.cut(qa[1]['question'])))
        questions.append(qa[1]['question'])
        answers.append(  qa[1]['answer'])
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    cs = cosine_similarity(X)
    rank = cs[0].argsort()[::-1]
    qa_dic = {}
    qa_dic['question'] = questions[rank[1]]
    qa_dic['answer']   = answers[rank[1]]
    qa_dic['similarity']   = cs[0][rank[1]]
    if qa_dic['similarity'] >= 0.7:
        return qa_dic
    else:
        qa_dic['answer'] = '你的問題現在對我而言有點複雜, 我還要持續學習, 等我以後變聰明以後再回答你'
        return qa_dic

In [137]:
getSimiliarAnswer('我好帥')

{'answer': '你的問題現在對我而言有點複雜, 我還要持續學習, 等我以後變聰明以後再回答你',
 'question': 'Q：只要需要長期看護，長期看護險就可以理賠？',
 'similarity': 0.0}

## 語音機器人回答

In [138]:
import speech_recognition

def listenToMe():
    r = speech_recognition.Recognizer()

    with speech_recognition.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
    return r.recognize_google(audio, language='zh-TW')

In [139]:
from gtts import gTTS
from pygame import mixer
mixer.init()

import tempfile
def speak(sentence):
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts = gTTS(text=sentence, lang='zh')
        tts.save("{}.mp3".format(fp.name))
        mixer.music.load('{}.mp3'.format(fp.name))
        mixer.music.play()

In [141]:
speak(getSimiliarAnswer(listenToMe())['answer'])

## 抓取富邦網路e方便

In [169]:
html_sample = '''
<select id="Qtype" name="Qtype">
                    <option value="-10">不分類</option>
                            <option value="06">熱門問題</option>
                            <option value="01">保費相關</option>
                            <option value="02">保單變更</option>
                            <option value="03">保單借款/還款</option>
                            <option value="04">理賠</option>
                            <option value="07">生存金/滿期金/紅利</option>
                            <option value="08">網路會員相關</option>
                            <option value="09">旅平險</option>
                            <option value="10">電子通知/保單</option>
                            <option value="05">其他</option>
                </select>
'''
soup = BeautifulSoup(html_sample, 'lxml')
type_dic = {}
for ele in soup.select('option'):
    type_dic[ele['value']] = ele.text

In [171]:
type_dic
del type_dic['-10']

In [172]:
type_dic

{'01': '保費相關',
 '02': '保單變更',
 '03': '保單借款/還款',
 '04': '理賠',
 '05': '其他',
 '06': '熱門問題',
 '07': '生存金/滿期金/紅利',
 '08': '網路會員相關',
 '09': '旅平險',
 '10': '電子通知/保單'}

In [162]:
def getQA(url):
    qa_dic = {}
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    question = soup.select_one('title').text
    answer   = ' '.join([' '.join(p.text.split()) for p in soup.select('h2')])
    qa_dic['question'] = question
    qa_dic['answer']   = answer
    qa_dic['link']     = url
    return qa_dic

In [183]:
import requests
from bs4 import BeautifulSoup
import time
payload = {
#'Qtype':'-10',
'allQuery':'N',
'toPage':'1',
'Count_per_Page':'300'    
}
qa_ary = []
for qtype in type_dic:
    payload['Qtype'] = qtype
    res = requests.post('https://me.fubonlife.com.tw/C03002000000_FAQ.choice.view', data = payload)
    soup = BeautifulSoup(res.text, 'lxml')
    domain = 'https://me.fubonlife.com.tw'
    
    for rec in soup.select('#formqa a'):
        if 'FlowID=' in rec['href']:
            qadic = getQA(domain + rec['href'])
            qadic['type'] = type_dic.get(qtype)
            qa_ary.append(qadic)
            #time.sleep(1)

In [185]:
import pandas
qa_df = pandas.DataFrame(qa_ary)
#qa_df
qa_df = qa_df[['type','question', 'answer', 'link']]
qa_df.to_excel('fuboneqa.xlsx')

## 根據相似度讓電腦自動回答問題

### 讀取知識庫

In [187]:
import pandas
qa_df = pandas.read_excel('fuboneqa.xlsx')

### 使用者問題

In [195]:
q = '醫生說我有牙周病的問題，請問是否可以理賠?'
q_seg = ' '.join(jieba.cut(q))

### 文章斷詞

In [196]:

corpus    = [q_seg]
answers   = ['NONE']
questions = [q]
for rec in qa_df.iterrows():
    corpus.append(' '.join(jieba.cut(rec[1]['question'])))
    questions.append(rec[1]['question'])
    answers.append(  rec[1]['answer']  )

### 建立詞頻矩陣

In [197]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<210x670 sparse matrix of type '<class 'numpy.int64'>'
	with 2292 stored elements in Compressed Sparse Row format>

### 計算相似度

In [199]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(X)

### 得到回傳答案

In [200]:
rank = cs[0].argsort()[::-1]
qa_dic = {}
qa_dic['question'] = questions[rank[1]]
qa_dic['answer']   = answers[rank[1]]
qa_dic['similarity']   = cs[0][rank[1]]
qa_dic

{'answer': '由於此類案件常涉及是否為牙齒治療手術及是否為除外責任問題，故提醒您所附的文件多留意以下幾點： 一、診斷證明書建議載明「國際疾病傷害及死因分類標準」編號(ICD code)。 二、診斷證明書建議載明手術名稱內容為何？ 三、建議檢附該次事故就診之完整病歷及X光片。 四、牙齒疾病之診斷工具為X光片，可檢附上述報告內容供理賠審核參考。 五、檢附詳細之醫療費用明細及收據(正本)。',
 'question': 'Fubon - 我要問-(Q)請問牙周病治療是否可以申請理賠？ (205)',
 'similarity': 0.50395263067896956}

In [202]:
def getSimiliarAnswer(knowledge,q, similarity):
    q_seg = ' '.join(jieba.cut(q))
    corpus    = [q_seg]
    questions = [q]
    answers   = ['NONE']
    for qa in knowledge.iterrows():
        corpus.append(' '.join(jieba.cut(qa[1]['question'])))
        questions.append(qa[1]['question'])
        answers.append(  qa[1]['answer'])
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    cs = cosine_similarity(X)
    rank = cs[0].argsort()[::-1]
    qa_dic = {}
    qa_dic['question'] = questions[rank[1]]
    qa_dic['answer']   = answers[rank[1]]
    qa_dic['similarity']   = cs[0][rank[1]]
    if qa_dic['similarity'] >= similarity:
        return qa_dic
    else:
        qa_dic['answer'] = '你的問題現在對我而言有點複雜, 我還要持續學習, 等我以後變聰明以後再回答你'
        return qa_dic

In [204]:
getSimiliarAnswer(qa_df, '醫生說我有牙周病的問題，請問是否可以理賠?', 0.5)

{'answer': '由於此類案件常涉及是否為牙齒治療手術及是否為除外責任問題，故提醒您所附的文件多留意以下幾點： 一、診斷證明書建議載明「國際疾病傷害及死因分類標準」編號(ICD code)。 二、診斷證明書建議載明手術名稱內容為何？ 三、建議檢附該次事故就診之完整病歷及X光片。 四、牙齒疾病之診斷工具為X光片，可檢附上述報告內容供理賠審核參考。 五、檢附詳細之醫療費用明細及收據(正本)。',
 'question': 'Fubon - 我要問-(Q)請問牙周病治療是否可以申請理賠？ (205)',
 'similarity': 0.50395263067896956}

In [209]:
speak(getSimiliarAnswer(qa_df,listenToMe(), 0.5)['answer'])

## 新聞分群

In [213]:
import requests
res = requests.get('https://raw.githubusercontent.com/ywchiu/fuboni/master/data/1435449602.xml')
with open('news.xml', 'w', encoding='utf-8') as f:
    f.write(res.text)

In [214]:
with open('news.xml', encoding='utf-8') as f:
    news = f.read()

In [217]:
from xml.dom import minidom
from xml.etree import ElementTree
import jieba.analyse

with open('news.xml', 'r', encoding='utf-8') as f:
    events=ElementTree.fromstring(f.read())

corpus = []
ary= []
for elem in events.findall('./channel/item'):
    title = elem.find('title').text
    description = elem.find('description').text
    ary.append(title)
    corpus.append(' '.join(jieba.cut(description)))

In [218]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<147x12838 sparse matrix of type '<class 'numpy.int64'>'
	with 23841 stored elements in Compressed Sparse Row format>

In [219]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(X)

In [220]:
cs.shape

(147, 147)

In [232]:
from sklearn import cluster
c = cluster.KMeans(n_clusters = 4, init = 'k-means++' )
k_data = c.fit_predict(cs)

In [233]:
k_data

array([0, 0, 2, 2, 0, 3, 0, 3, 3, 2, 2, 0, 3, 1, 1, 2, 0, 3, 3, 2, 2, 0, 2,
       3, 2, 0, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 0, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 0, 3, 3, 3, 2, 3, 0, 0, 0, 2, 0,
       3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 0, 2, 0, 0, 0, 3, 3, 1, 1, 1, 0, 2, 3,
       3, 1, 0, 0, 1, 0, 1, 3, 0, 1, 1, 1, 0, 1, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       3, 3, 3, 3, 1, 0, 0, 1, 1])

In [237]:
np_titles = np.array(ary)
for t in np_titles[k_data ==0, ]:
    print(t)

八仙塵爆  五相關人依公共危險重傷害法辦
八仙樂園意外 病患持續增加中
6月28日各報頭版要聞
八仙樂園舞台大火 逾400人輕重傷
八仙樂園意外 毛揆取消視察臺東行程
八仙樂園貼出暫停營業海報
八仙樂園爆炸案 專家認玉米粉危險性高應管制使用
八仙樂園粉塵燃爆 400人傷 負責人被移送
八仙案  新店慈濟4傷者危急
八仙樂園爆炸案 負責人等5人移送
侯友宜探視傷者：追究責任
八仙樂園火警  國軍加入救援
八仙樂園爆炸案災害應變中心　1時30分一級開設
彩色派對主辦人：風勢太大，引燃粉塵
八仙樂園大火  傷者查詢電話
派對彩粉首次釀禍  蔣丙煌：研議管理
八仙樂園火警受傷名單一覽表
00：35統計　八仙大火重傷97人、輕傷132人
衛福部長：北北基醫護人員全力動員
八仙樂園粉塵瞬燃　還原失事現場影片曝光
八仙意外毛揆取消台東熱氣球活動
八仙樂園爆炸案 士檢派3檢察官現場指揮
八仙爆炸案   陳國恩：全力支援
八仙大火傷患爆量  衛福調度醫院
八仙樂園意外 洪秀柱：為傷者祈福
陳國恩指示警方全力動員 協助救護交通順暢
八仙樂園爆炸意外 朱立倫：立即停園接受調查
北榮33名八里傷患  2/3傷重
八仙樂園塵爆215傷　朱立倫下令即刻停業
八仙樂園爆炸 朱立倫4點指示
【八仙意外】馬英九、毛治國第一時間得知　指示全
【八仙意外】讓專業的來！柯文哲指示啟動EOC
八仙水上樂園爆炸逾百人傷　三軍總醫院收治傷患


## 新聞分群2

In [239]:
import pandas
newsdf = pandas.read_excel('data/appledaily20171214.xlsx')

In [240]:
newsdf.head()

,category,content,link,title
0,政治,新增：立委說法民進黨立法院黨團預計在明天的院會中，讓改制農田水利會的《農田水利會組織通則》修...,https://tw.news.appledaily.com/politics/realti...,【更新】水利會改官派明闖關 綠委24小時前顧議場大門防藍突襲
1,論壇,邱俊棠／台中市民、見習醫師；曾任台灣醫學生聯合會對外副會長對於公民參與公眾事務而能得到單位首...,https://tw.news.appledaily.com/forum/realtime/...,請中市府為所當為 加速中火燃煤限制
2,社會,被控來台涉發展情報組織的中國學生周泓旭，因接觸我方外交部官員而露餡落網，今年9月被台北地院一...,https://tw.news.appledaily.com/local/realtime/...,陸生共諜嗆台司法 「不敢公開審理我」
3,生活,【廣編特輯】 對於全能神教會在中國所遭受的迫害與在韓國、香港及臺灣所面對的反面宣傳，非政府組...,https://tw.news.appledaily.com/life/realtime/2...,【特企】NGO聯名譴責中共迫害宗教信仰
4,生活,立法院交通委員會今審查《道路交通管理處罰條例》，通過營業大客車駕駛人行駛一般道路未繫安全帶，...,https://tw.news.appledaily.com/life/realtime/2...,乘客搭大客車不配合繫安全帶 立院初審通過可罰6千元


In [241]:
corpus   = []
titles   = []
contents = []
for elem in newsdf.iterrows():
    titles.append(elem[1]['title'])
    contents.append(elem[1]['content'])
    corpus.append(' '.join(jieba.cut(elem[1]['content'])))

In [242]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<899x38624 sparse matrix of type '<class 'numpy.int64'>'
	with 128131 stored elements in Compressed Sparse Row format>

In [243]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(X)

### KMeans 分群

In [255]:
from sklearn import cluster
c = cluster.KMeans(n_clusters = 20, init = 'k-means++' )
k_data = c.fit_predict(cs)

In [256]:
np_titles = np.array(titles)

In [265]:
for title in np_titles[k_data == 0]:
    print(title)

小嫻婚變冒毒菇　勾于美人「奉茶」夢魘
「想抱孫要看天！」　他批小嫻婆婆殘忍強逼
【好聚不好散】離婚掏出6千萬　這女星比小嫻還慘
【狗仔偷拍】小嫻搬離何守正家租66坪房　月租6萬元
許聖梅心疼小嫻被當空氣　爆何守正「有兩個女學員」
【動畫解盤】毒菇跳火線譙seafood　小嫻難瘦香菇
十二月十四日各報頭條搶先報
不捨善良小嫻慘遭婚變　乃哥「命運捉弄人」
【獨家】小嫻賣房求子　婆婆竟拒入籍何家
小嫻離婚導火線　拉何守正信妙禪
【狗仔偷拍】何守正現身！「全台最沒尊嚴的婆婆」也出來了
小嫻中分手魔咒！ 同公司4女星全都婚變
何守正姐姐神護航！遭網友酸：全台最討人厭大姑
小嫻別傻傻被欺負！女律師說「姐寶」就要這樣對付
教友小嫻婚姻觸礁　曾之喬談情避不開Seafood
小嫻守正結婚在台沒登記　想離婚只有兩條路
胡瓜2個月前耳聞小嫻婚變　震驚之餘好心疼
昔日搭檔談小嫻婚變　曾國城這樣說
何守正兩個姊姊護航扯婆媳　「他」戳破媽寶特色
大姑出面護弟！轟小嫻不能生「媽媽是全台最沒有尊嚴的婆婆」
小嫻信奉妙禪　關鍵原因與何守正有關！
小嫻何守正想離婚　必須先做這件事！
女網紅因為這理由挺余祥銓！讓人不知該哭還是笑
「小嫻不快樂！」　許聖梅：何守正虧欠她
【內幕動畫】小嫻婚變何守正姊反擊　不滿媽煮飯侍奉星媳婦
小嫻多信妙禪？　曾見證「師父帶我跳舞」
【話當年】被拍和她上賓館　何守正掰了阿妹
小嫻婚變無徵兆　男星嘆：兩人向來出雙入對
【獨家內幕】太傷！小嫻被分手　何守正當小三面前攤牌
【小嫻離婚】何守正稱沒有遺憾　人妻女星超火「一嘴屁話」
【小嫻離婚】3大退讓人財兩失　求子花光430萬積蓄


## 社群偵測

In [269]:
m = cs > 0.5
m = m.astype(int)

In [270]:
m

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

### 安裝 Networkx

In [271]:
! pip install networkx

In [273]:
import networkx as nx
from scipy.sparse import csr_matrix
G = nx.from_scipy_sparse_matrix(csr_matrix(m))

### 使用Python Louvain

In [278]:
! pip install python-louvain

### 使用社群偵測法

In [281]:
import community
k_data = community.best_partition(G)

In [346]:
from collections import Counter
c = Counter(k_data.values())
c.most_common(10)

[(3, 30),
 (70, 13),
 (114, 7),
 (201, 4),
 (341, 4),
 (17, 3),
 (45, 3),
 (185, 3),
 (246, 3),
 (524, 3)]

In [305]:
len(set(list(k_data.values())))

791

In [345]:
grp = np.array(list(k_data.values()))
for ele in np_titles[grp==341]:
    print(ele)

【壹週刊】百億貸款將到期　柯P放容積解套京華城
560%容積爭議　威京：只是拿回應有的
京華城容積率增至560%案　北市都發局同意、待都委會審議
及時雨！柯文哲放寬容積　京華城爽納百億


In [348]:
for k,v in c.most_common(10):
    print('group: {}'.format(k) )
    for ele in np_titles[grp==k]:
        print(ele)
    print('----------------------')

group: 3
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
【特企】NGO聯名譴責中共迫害宗教信仰
----------------------
group: 70
【狗仔偷拍】小嫻搬離何守正家租66坪房　月租6萬元
許聖梅心疼小嫻被當空氣　爆何守正「有兩個女學員」
【動畫解盤】毒菇跳火線譙seafood　小嫻難瘦香菇
【獨家】小嫻賣房求子　婆婆竟拒入籍何家
小心！在美結婚台灣沒登記　偷腥照樣能捉姦
小嫻離婚導火線　拉何守正信妙禪
小嫻守正結婚在台沒登記　想離婚只有兩條路
小嫻何守正想離婚　必須先做這件事！
【內幕動畫】小嫻婚變何守正姊反擊　不滿媽煮飯侍奉星媳婦
小嫻婚變無徵兆　男星嘆：兩人向來出雙入對
【獨家內幕】太傷！小嫻被分手　何守正當小三面前攤牌
【小嫻離婚】何守正稱沒有遺憾　人妻女星超火「一嘴屁話」
【小嫻離婚】3大退讓人財兩失　求子花光430萬積蓄
----------------------
group: 114
驚！　野生捕獲恐龍遛汪星人
驚！　野生捕獲

## 富邦人壽問答集分群

In [349]:
import pandas
qa_df = pandas.read_excel('fuboneqa.xlsx')

In [350]:
qa_df.head()

,type,question,answer,link
0,熱門問題,Fubon - 我要問-(熱門)什麼是電子通知單？「用行動愛地球」活動內容是什麼？ (207),一、配合數位化時代來臨，故以電子郵件方式寄送各項送金單或通知單，並取消紙本表單/單據寄送。 ...,https://me.fubonlife.com.tw/C03002000000_FAQ.v...
1,熱門問題,Fubon - 我要問-(熱門)請問如何申請電子通知單/申請電子收據/电子通知单/申請e帳單...,一、申請時間：保單有效期間內，可隨時辦理。 二、應注意事項： (一)申請電子通知單服務，需同...,https://me.fubonlife.com.tw/C03002000000_FAQ.v...
2,熱門問題,Fubon - 我要問-(熱門)如何申請/加入網路e方便會員？包含哪些服務？ (204),【人壽保戶會員網路e方便線上交易專區】中，會員服務及申請有以下兩種，說明如下： 一、查詢會員...,https://me.fubonlife.com.tw/C03002000000_FAQ.v...
3,熱門問題,Fubon - 我要問-(熱門)請問您要查詢的是車險/強制險/十全大補/產險/汽車險商品嗎？...,因富邦人壽與富邦產險為富邦金控不同之子公司，若您是要查詢： 富邦產物保險/綜合險/意外產險/...,https://me.fubonlife.com.tw/C03002000000_FAQ.v...
4,熱門問題,Fubon - 我要問-(熱門)如何申請理賠？ (206),各項保險金自得為請求之日起須於兩年內提出申請，請於保險事故發生後儘速申請以免影響權利。 連結...,https://me.fubonlife.com.tw/C03002000000_FAQ.v...


In [353]:
corpus   = []
questions   = []
answers = []
for elem in qa_df.iterrows():
    questions.append(elem[1]['question'])
    answers.append(elem[1]['answer'])
    corpus.append(' '.join(jieba.cut(elem[1]['question'])))

In [364]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
X

<209x669 sparse matrix of type '<class 'numpy.float64'>'
	with 2285 stored elements in Compressed Sparse Row format>

In [365]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(X)

In [366]:
m = cs > 0.5
m = m.astype(int)

In [367]:
import networkx as nx
from scipy.sparse import csr_matrix
G = nx.from_scipy_sparse_matrix(csr_matrix(m))

In [368]:
import community
k_data = community.best_partition(G)

In [369]:
from collections import Counter
c = Counter(k_data.values())
c.most_common(10)

[(19, 4),
 (53, 4),
 (87, 4),
 (2, 3),
 (27, 3),
 (72, 3),
 (0, 2),
 (1, 2),
 (3, 2),
 (4, 2)]

In [370]:
np_questions = np.array(questions)
grp = np.array(list(k_data.values()))

In [371]:
for k,v in c.most_common(10):
    print('group: {}'.format(k) )
    for ele in np_questions[grp==k]:
        print(ele)
    print('----------------------')

group: 19
Fubon - 我要問-金融機構轉帳扣款失敗怎麼辦？ (203)
Fubon - 我要問-信用卡扣款失敗怎麼辦？ (208)
Fubon - 我要問-現在已是金融機構轉帳代繳保費，要如何變更轉帳銀行或扣款帳戶？ (203)
Fubon - 我要問-金融機構轉帳或信用卡扣款前，會寄繳費通知單嗎？ (207)
----------------------
group: 53
Fubon - 我要問-(Q)請問您欲還款，需要查詢您的「保單借款金額」嗎？ (205)
Fubon - 我要問-(Q)我有保單借款還款/保單還款/保單貸款還款/保單還款記錄/保單還款確認，怎麼查詢? (201)
Fubon - 我要問-我有保單借款，如何還款? (201)
Fubon - 我要問-請問臨櫃是否可辦理保單借款的還款？如果要以現金方式還款，有沒有金額上的限制？ (202)
----------------------
group: 87
Fubon - 我要問-安泰住院醫療定期保險附約(HSR),於精神病住院是否給付？ (205)
Fubon - 我要問-中醫門診看鼻炎,投保"安泰住院醫療定期保險附約(HSR)"及"安泰住院醫療保險附約(THS)"是否給付？ (206)
Fubon - 我要問-安泰住院醫療定期保險附約(HSR)以自費身份就醫如何計算給付？ (208)
Fubon - 我要問-門診手術是否能申請"安泰住院醫療定期保險附約(HSR)"及"安泰住院醫療保險附約"(THS)之理賠給付？ (201)
----------------------
group: 2
Fubon - 我要問-(熱門)如何申請/加入網路e方便會員？包含哪些服務？ (204)
Fubon - 我要問-(Q)如何申請加入富邦人壽網路e方便會員? (204)
Fubon - 我要問-(Q)如何申請/加入網路e方便會員？包含哪些服務？ (204)
----------------------
group: 27
Fubon - 我要問-自動墊繳是否有保險費繳納證明？ (202)
Fubon - 我要問-保險費已繳納，為何又收到保險費提醒繳納通知書？ (203)
Fubon - 我要問-什麼是「保險費繳納證明」？ (204)
----------------------
group: 72


## 新聞分類

In [421]:
import pandas
newsdf = pandas.read_excel('data/appledaily20171214.xlsx')

corpus     = []
titles     = []
categories = []

for rec in newsdf[newsdf['category'].isin(['財經地產', '社會', '國際']) ].iterrows():
    corpus.append(' '.join(jieba.cut(rec[1]['content'])))
    titles.append(rec[1]['title'])
    categories.append(rec[1]['category'])

In [422]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [423]:
X

<383x20211 sparse matrix of type '<class 'numpy.int64'>'
	with 50916 stored elements in Compressed Sparse Row format>

In [424]:
len(categories)

383

In [425]:
from sklearn.cross_validation import train_test_split
train_data, test_data, train_tag, test_tag, train_titles, test_titles = train_test_split(X, categories, titles, test_size = 0.33, random_state = 123)

In [426]:
train_data.shape

(256, 20211)

In [427]:
test_data.shape

(127, 20211)

In [428]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.01)
clf.fit(train_data, train_tag)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [429]:
predicted = clf.predict(test_data)

In [430]:
sum(predicted == test_tag) / len(test_tag)

0.92125984251968507

In [431]:
from sklearn.metrics import confusion_matrix
print(clf.classes_)
confusion_matrix(test_tag, predicted)

['國際' '社會' '財經地產']


array([[39,  3,  1],
       [ 3, 49,  1],
       [ 2,  0, 29]])

In [432]:
np_test_titles = np.array(test_titles)
np_test_tags = np.array(test_tag)
np_test_tags[test_tag !=  predicted]
np_test_titles[test_tag !=  predicted]

array(['5歲男童被水管絆倒\u3000右手指遭清潔機皮帶絞斷', '小心！在美結婚台灣沒登記\u3000偷腥照樣能捉姦',
       '女高中生遭霸凌跳樓爆頭\u3000一張照片令家屬心碎了', '玉山清晨降雪\u3000陽光露臉融化糖霜潤大地',
       '浩鼎案開庭\u3000翁啟惠：美事一樁被誤會', '【霸氣貼背】「這是我們的夢幻車款」！Kia新車北美試駕超給力',
       '老謝：美菲2位狂人兄弟 今年交出不錯成績單\u3000', '20歲女搭電車時睡著\u3000男見狀上前強吻',
       'friDay影音攜手ELEVEN體育家族\u3000直播多元賽事', '見義勇為\u3000青年4度衝進火場救人\u3000'], 
      dtype='<U36')

## 問題分類

In [433]:
import pandas
qa_df = pandas.read_excel('fuboneqa.xlsx')

In [435]:
qa_df = qa_df[qa_df['type']!= '熱門問題']

In [444]:
qa_df['type'].value_counts()

理賠            58
其他            52
保單變更          28
保費相關          19
網路會員相關        13
保單借款/還款       12
生存金/滿期金/紅利     5
旅平險            4
電子通知/保單        3
Name: type, dtype: int64

In [445]:
corpus    = []
questions = []
answers   = []
types     = []
for elem in qa_df[qa_df['type'].isin(['理賠', '保單變更']) ].iterrows():
    questions.append(elem[1]['question'])
    answers.append(elem[1]['answer'])
    corpus.append(' '.join(jieba.cut(elem[1]['question'])))
    types.append(elem[1]['type'])

In [446]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [447]:
from sklearn.cross_validation import train_test_split
train_data, test_data, train_types, test_types, train_questions, test_questions = train_test_split(X, types, questions, test_size = 0.33, random_state = 123)

In [448]:
train_data.shape

(57, 347)

In [449]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.01)
clf.fit(train_data, train_types)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [450]:
predicted = clf.predict(test_data)

In [454]:
from sklearn.metrics import accuracy_score
accuracy_score(test_types, predicted)

0.89655172413793105

In [452]:
from sklearn.metrics import confusion_matrix
print(clf.classes_)
confusion_matrix(test_types, predicted)

['保單變更' '理賠']


array([[ 7,  0],
       [ 3, 19]])

In [453]:
np_test_questions = np.array(test_questions)
np_test_types = np.array(test_types)
#np_test_tags[test_tag !=  predicted]
np_test_questions[test_types !=  predicted]

array(['Fubon - 我要問-收據遺失是否可以副本或補發收據代替申請？ (201)',
       'Fubon - 我要問-(Q)請問哪裡可以下載【理賠類】表單/表格/申請書/聲明書？ (204)',
       'Fubon - 我要問-(Q)詢問雷射/運動傷害/腰間盤突出手術/殘廢等級/眼睛換水晶球/痔瘡開刀/骨刺/氣管內管置入術/根管治療/胃鏡/前十字韌帶板機指/冷凍/白內障手術/甲狀腺/心臟支架/子宮長水瘤/大腸鏡息肉可否申請? (206)'], 
      dtype='<U118')